In [4]:
import sys
sys.path.append('../statistical_calculations')  # Adds the statistical_calculations directory to the path
import disproportionaly_analysis
from tqdm import tqdm
import json
from pymongo import MongoClient

# Connect to your MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client['vaers']  
collection = db['combinations cleaned']

## create adverse reaction list for each manufacturer

### PFIZER/BIONTECH

In [5]:
Pfizer_adverse_reactions = [
    "Vomiting",
    "Lymphadenopathy",
    "Rash", 
    "Pruritus", 
    "Urticaria", 
    "Angioedema",
    "Anaphylaxis",
    "Decreased appetite",
    "Insomnia",
    "Headache",
    "Dizziness",
    "Lethargy",
    #"Acute peripheral facial paralysis",
    "Facial paralysis",
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Diarrhoea",
    "Nausea",
    "Hyperhidrosis",
    "Night sweats",
    "Erythema multiforme",
    "Arthralgia",
    "Myalgia",
    "Pain in extremity",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site redness",
    "Asthenia",
    "Malaise",
    "Injection site pruritus",
    "Extensive swelling of vaccinated limb",
    "Facial swelling"
]

In [6]:
len(Pfizer_adverse_reactions)

37

In [7]:
from tqdm import tqdm
import disproportionaly_analysis

signals_Pfizer = []

for reaction in Pfizer_adverse_reactions:
    query = {
      "vaccine": "COVID19",
      "manufacturer": "PFIZER\BIONTECH",
      "symptom": reaction   
    }
    
    results = list(collection.find(query))
    
    if not results:
        result_dict = {
            'symptom': reaction,
            'status': "Not found",
        }
        signals_Pfizer.append(result_dict)
        continue
    
    for result in tqdm(results):
        DE = result.get("DE")
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")
        
        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue
        
        contingency_table = [
            [DE, dE],
            [De, de]
        ]
        
        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
        
        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
        
        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
        
        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
        
        # Determine if the reaction meets criteria for a signal
        method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
        method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and ror_ci_lower > 1
        
        
        if method_1 or method_2:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Detected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Pfizer.append(result_dict)
        else:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Undetected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Pfizer.append(result_dict)


100%|██████████| 1/1 [00:00<00:00, 4675.92it/s]


In [8]:
count_method_1_true = sum(entry['Chi-square_Test'] for entry in detected_signals_Pfizer_product_info)
count_method_2_true = sum(entry['CI_Test'] for entry in detected_signals_Pfizer_product_info)
count_both_true = sum(entry['Chi-square_Test'] and entry['CI_Test'] for entry in detected_signals_Pfizer_product_info)

print("Number of entries where both method_1 and method_2 are true:", count_both_true)
print("Number of entries where method_1 is true:", count_method_1_true)
print("Number of entries where method_2 is true:", count_method_2_true)

NameError: name 'detected_signals_Pfizer_product_info' is not defined

In [9]:
file_name_filtered = "Arrays/Signals_Pfizer.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_Pfizer, f)

In [10]:
detected_count = 0
undetected_count = 0
not_found_count = 0

for result in signals_Pfizer:
    status = result['status']
    if status == 'Detected':
        detected_count += 1
    elif status == 'Undetected':
        undetected_count += 1
    elif status == 'Not found':
        not_found_count += 1

print("Detected count:", detected_count)
print("Undetected count:", undetected_count)
print("Not found count:", not_found_count)


Detected count: 23
Undetected count: 11
Not found count: 3


### MODERNA

In [11]:
Moderna_adverse_reactions = [
    "Lymphadenopathy",
    "Anaphylaxis",
    "Hypersensitivity",
    "Decreased appetite",
    "Irritability", 
    "Crying",
    "Headache",
    "Sleepiness",
    "Dizziness",
    #"Acute peripheral facial paralysis",
    "Facial paralysis", #instead of facial paralysis
    "Hypoaesthesia",
    "Paraesthesia",
    "Myocarditis",
    "Pericarditis",
    "Nausea", 
    "Vomiting",
    "Diarrhoea",
    "Abdominal pain",
    "Rash",
    "Urticaria",
    "Erythema multiforme",
    "Mechanical urticaria",
    "Chronic urticaria",
    "Myalgia",
    "Arthralgia",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site erythema",
    "Injection site urticaria",
    "Injection site rash",
    "Delayed injection site reaction",
    "Injection site pruritus",
    "Facial swelling",
    "Extensive swelling of vaccinated limb"
]


In [12]:
len(Moderna_adverse_reactions)

38

In [13]:
from tqdm import tqdm
import disproportionaly_analysis

signals_Moderna = []

for reaction in Moderna_adverse_reactions:
    query = {
      "vaccine": "COVID19",
      "manufacturer": "MODERNA",
      "symptom": reaction   
    }
    
    results = list(collection.find(query))
    
    if not results:
        result_dict = {
            'symptom': reaction,
            'status': "Not found",
        }
        signals_Moderna.append(result_dict)
        continue
    
    for result in tqdm(results):
        DE = result.get("DE")
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")
        
        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue
        
        contingency_table = [
            [DE, dE],
            [De, de]
        ]
        
        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
        
        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
        
        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
        
        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
        
        # Determine if the reaction meets criteria for a signal
        method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
        method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and ror_ci_lower > 1
        
        
        if method_1 or method_2:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Detected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Moderna.append(result_dict)
        else:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Undetected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Moderna.append(result_dict)


100%|██████████| 1/1 [00:00<00:00, 5637.51it/s]


In [14]:
count_method_1_true = sum(entry['Chi-square_Test'] for entry in detected_signals_Moderna_product_info)
count_method_2_true = sum(entry['CI_Test'] for entry in detected_signals_Moderna_product_info)
count_both_true = sum(entry['Chi-square_Test'] and entry['CI_Test'] for entry in detected_signals_Moderna_product_info)

print("Number of entries where both method_1 and method_2 are true:", count_both_true)
print("Number of entries where method_1 is true:", count_method_1_true)
print("Number of entries where method_2 is true:", count_method_2_true)

NameError: name 'detected_signals_Moderna_product_info' is not defined

In [15]:
file_name_filtered = "Arrays/Signals_Moderna.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_Moderna, f)

In [16]:
detected_count = 0
undetected_count = 0
not_found_count = 0

for result in signals_Moderna:
    status = result['status']
    if status == 'Detected':
        detected_count += 1
    elif status == 'Undetected':
        undetected_count += 1
    elif status == 'Not found':
        not_found_count += 1

print("Detected count:", detected_count)
print("Undetected count:", undetected_count)
print("Not found count:", not_found_count)


Detected count: 22
Undetected count: 11
Not found count: 5


### JANSSEN

In [17]:
Janssen_adverse_reactions = [
    "Lymphadenopathy",
    "Immune thrombocytopenia",
    "Anaphylaxis",
    "Headache",
    "Dizziness",
    "Tremor",
    "Urticaria",
    "Hypersensitivity",
    "Paraesthesia",
    "Hypoaesthesia",
    "Facial paralysis",
    "Tinnitus",
    "Guillain-Barre syndrome",
    #"Transverse myelitis", not in DB
    "Myelitis", #instead of Transverse myelitis
    "Venous thromboembolism",
    "Thrombosis in combination with thrombocytopenia",
    "Myocarditis", 
    "Pericarditis",
    "Capillary leak syndrome",
    #"Small vessel cutaneous vasculitis", not in DB
    "Cutaneous vasculitis", #instead of Small vessel cutaneous vasculitis"
    "Nausea",
    "Cough", 
    "Oropharyngeal pain",
    "Sneezing",
    "Diarrhoea",
    "Vomiting",
    "Rash",
    "Hyperhidrosis",
    "Myalgia",
    "Arthralgia",
    "Muscular weakness",
    "Back pain",
    "Pain in extremity",
    "Fatigue",
    "Injection site pain",
    "Injection site swelling",
    "Chills",
    "Pyrexia"
]

In [18]:
len(Janssen_adverse_reactions)

38

In [19]:
from tqdm import tqdm
import disproportionaly_analysis

signals_Janssen = []

for reaction in Janssen_adverse_reactions:
    query = {
      "vaccine": "COVID19",
      "manufacturer": "JANSSEN",
      "symptom": reaction   
    }
    
    results = list(collection.find(query))
    
    if not results:
        result_dict = {
            'symptom': reaction,
            'status': "Not found",
        }
        signals_Janssen.append(result_dict)
        continue
    
    for result in tqdm(results):
        DE = result.get("DE")
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")
        
        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue
        
        contingency_table = [
            [DE, dE],
            [De, de]
        ]
        
        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
        
        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
        
        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
        
        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
        
        # Determine if the reaction meets criteria for a signal
        method_1 = rrr > 2 and prr > 2 and ror > 2  and IC >0 and chi_square > 3.841
        method_2 = IC_ci_lower > 0 #and ror_ci_lower > 1 and prr_ci_lower > 1 and ror_ci_lower > 1
        
        
        if method_1 or method_2:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Detected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Janssen.append(result_dict)
        else:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Undetected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Janssen.append(result_dict)


100%|██████████| 1/1 [00:00<00:00, 5745.62it/s]


In [20]:
count_method_1_true = sum(entry['Chi-square_Test'] for entry in detected_signals_Janssen_product_info)
count_method_2_true = sum(entry['CI_Test'] for entry in detected_signals_Janssen_product_info)
count_both_true = sum(entry['Chi-square_Test'] and entry['CI_Test'] for entry in detected_signals_Janssen_product_info)

print("Number of entries where both method_1 and method_2 are true:", count_both_true)
print("Number of entries where method_1 is true:", count_method_1_true)
print("Number of entries where method_2 is true:", count_method_2_true)

NameError: name 'detected_signals_Janssen_product_info' is not defined

In [21]:
file_name_filtered = "Arrays/Signals_Janssen.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_Janssen, f)

In [22]:
detected_count = 0
undetected_count = 0
not_found_count = 0

for result in signals_Janssen:
    status = result['status']
    if status == 'Detected':
        detected_count += 1
    elif status == 'Undetected':
        undetected_count += 1
    elif status == 'Not found':
        not_found_count += 1

print("Detected count:", detected_count)
print("Undetected count:", undetected_count)
print("Not found count:", not_found_count)


Detected count: 24
Undetected count: 11
Not found count: 3


### NOVAVAX

In [23]:
Novavax_adverse_reactions = [
    "Headache",
    "Nausea",
    "Vomiting",
    "Myalgia",
    "Arthralgia",
    "Injection site tenderness",
    "Injection site pain",
    "Fatigue",
    "Malaise",
    "Injection site redness",
    "Injection site swelling",
    "Pyrexia",
    "Pain in extremity",
    "Injection site pruritus",
    "Chills",
    "Injection site warmth",
    "Lymphadenopathy",
    "Anaphylaxis",
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Hypertension",
    "Rash",
    "Erythema",
    "Pruritus",
    "Urticaria"
]


In [24]:
len(Novavax_adverse_reactions)

27

In [25]:
from tqdm import tqdm
import disproportionaly_analysis

signals_Novavax = []

for reaction in Novavax_adverse_reactions:
    query = {
      "vaccine": "COVID19",
      "manufacturer": "NOVAVAX",
      "symptom": reaction   
    }
    
    results = list(collection.find(query))
    
    if not results:
        result_dict = {
            'symptom': reaction,
            'status': "Not found",
        }
        signals_Novavax.append(result_dict)
        continue
    
    for result in tqdm(results):
        DE = result.get("DE")
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")
        
        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue
        
        contingency_table = [
            [DE, dE],
            [De, de]
        ]
        
        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
        
        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
        
        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
        
        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
        
        # Determine if the reaction meets criteria for a signal
        method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
        method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and ror_ci_lower > 1
        
        
        if method_1 or method_2:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Detected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Novavax.append(result_dict)
        else:
            result_dict = {
                'vaccine': result.get("vaccine"),
                'manufacturer': result.get("manufacturer"),
                'symptom': result.get("symptom"),
                'status': "Undetected",
                'DE': DE,
                'dE': dE,
                'De': De,
                'de': de,
                'rrr': [rrr, [rrr_ci_lower, rrr_ci_upper]],
                'prr': [prr, [prr_ci_lower, prr_ci_upper]],
                'ror': [ror, [ror_ci_lower, ror_ci_upper]],
                'chi_square': chi_square,
                'IC': [IC, [IC_ci_lower, IC_ci_upper]],
                'Chi-square_Test': method_1,
                'CI_Test': method_2  
            }
            signals_Novavax.append(result_dict)


100%|██████████| 1/1 [00:00<00:00, 4524.60it/s]


In [26]:
signals_Novavax

[{'vaccine': 'COVID19',
  'manufacturer': 'NOVAVAX',
  'symptom': 'Headache',
  'status': 'Undetected',
  'DE': 47,
  'dE': 291858,
  'De': 387,
  'de': 2186669,
  'rrr': [0.9196790398871169, [0.7020691024630864, 1.204738299179261]],
  'prr': [0.9196661052232553, [0.7020592259332376, 1.2047213594725152]],
  'ror': [0.9099097924209117, [0.6722042259348379, 1.2316730517307477]],
  'chi_square': 0.28825914915324086,
  'IC': [-0.1207976341020595, [-0.1689488361595891, -0.07264643204452988]],
  'Chi-square_Test': False,
  'CI_Test': False},
 {'vaccine': 'COVID19',
  'manufacturer': 'NOVAVAX',
  'symptom': 'Nausea',
  'status': 'Undetected',
  'DE': 36,
  'dE': 166602,
  'De': 398,
  'de': 2311925,
  'rrr': [1.2339808530016472, [0.902475272967593, 1.6872581345830975]],
  'prr': [1.2340314124829743, [0.9025122429888586, 1.6873272787431028]],
  'ror': [1.2552000829588212, [0.8923810936384158, 1.7655318557188304]],
  'chi_square': 1.470886104105226,
  'IC': [0.3033200091600391, [0.2422720903571

In [27]:
count_method_1_true = sum(entry['Chi-square_Test'] for entry in detected_signals_Novavax_product_info)
count_method_2_true = sum(entry['CI_Test'] for entry in detected_signals_Novavax_product_info)
count_both_true = sum(entry['Chi-square_Test'] and entry['CI_Test'] for entry in detected_signals_Novavax_product_info)

print("Number of entries where both method_1 and method_2 are true:", count_both_true)
print("Number of entries where method_1 is true:", count_method_1_true)
print("Number of entries where method_2 is true:", count_method_2_true)

NameError: name 'detected_signals_Novavax_product_info' is not defined

In [28]:
file_name_filtered = "Arrays/Signals_Novavax.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_Novavax, f)

In [29]:
detected_count = 0
undetected_count = 0
not_found_count = 0

for result in signals_Novavax:
    status = result['status']
    if status == 'Detected':
        detected_count += 1
    elif status == 'Undetected':
        undetected_count += 1
    elif status == 'Not found':
        not_found_count += 1

print("Detected count:", detected_count)
print("Undetected count:", undetected_count)
print("Not found count:", not_found_count)


Detected count: 3
Undetected count: 20
Not found count: 4
